In [1]:
import numpy as np
import scipy as sp

# Cinemática Directa IRB140

In [2]:
#Se determina la matriz A^6_0 y se determina el vector de configuración/signo conf. Recibe parámetros DH y vector q.

def pcd_IRB140(q, a, d, alfa):
    
    POSE = np.eye(4)
    conf = np.empty(3)
    
    for i in range(0,6):
        
        #Matriz genérica
        A = np.matrix([[np.cos(q[i]), -np.sin(q[i])*np.cos(alfa[i]), np.sin(q[i])*np.sin(alfa[i]), a[i]*np.cos(q[i])],
                      [np.sin(q[i]), np.cos(q[i])*np.cos(alfa[i]), -np.cos(q[i])*np.sin(alfa[i]), a[i]*np.sin(q[i])],
                      [0, np.sin(alfa[i]), np.cos(alfa[i]), d[i]],
                      [0, 0, 0, 1]])
        
        POSE = POSE.dot(A)
        
    
    #Calculo del vector de configuracion
    
    conf[0] = np.sign(d[3]*np.sin(q[1] + q[2]) + a[1]*np.cos(q[1]) + a[0])
    conf[1] = np.sign(np.cos(q[2]))
    conf[2] = np.sign(q[4])
    conf[conf == 0] = 1
    
    np.set_printoptions(suppress=True) #Suprime la notación científica

        
    return(POSE, conf)

# Cinemática Inversa IRB140

In [3]:
def pci_IRB140(POSE, a, d, alfa, conf, q1_actual, q4_actual):
    
    #Inicializo el vector de variables articulares
    q = np.zeros(6)
    #print(q)
    
    #Resuelvo el problema de posición
    px = POSE.item((0,3))
    py = POSE.item((1,3))
    pz = POSE.item((2,3))
    
    #Cálculo de q1 (q[0])
    s1 = conf[0]*py
    c1 = conf[0]*px
    
    #Acá hay problema con la indeterminación en (0,0) que es la posición sobre el eje z0.
    if (px == 0) and (py == 0):
        q[0] = q1_actual
    else:
        q[0] = np.arctan2(s1,c1)
    
    
    #Cálculo de q3 Codo
    
    s3 = ((px*np.cos(q[0]) + py*np.sin(q[0]) - a[0])**2 + pz**2 - a[1]**2 - d[3]**2)/(2*a[1]*d[3])
    
    if np.absolute(s3) > 1 :
        print("El punto no es alcanzable")
        return 0
        
    c3 = conf[1]*np.sqrt(1-(s3**2))
    q[2] = np.arctan2(s3,c3)
    
    
    #Cálculo de q2
    
    s2 = ((px*np.cos(q[0]) + py*np.sin(q[0]) - a[0])*(d[3]*np.cos(q[2]))) - (pz*(d[3]*np.sin(q[2]) + a[1]));
    c2 = ((px*np.cos(q[0]) + py*np.sin(q[0]) - a[0])*((d[3]*np.sin(q[2])) + a[1])) + (pz*d[3]*np.cos(q[2]));
    q[1] = np.arctan2(s2,c2)
    
    
    #Resuelvo la orientación calculando R^6_3
    
    R_3_0 = np.eye(3)
    
    for i in range(0,3):
        #Esto que viene ahora sale del problema directo porque toma sólo la de rotación 3x3
        A = np.matrix([[np.cos(q[i]), -np.sin(q[i])*np.cos(alfa[i]), np.sin(q[i])*np.sin(alfa[i]), a[i]*np.cos(q[i])],
                      [np.sin(q[i]), np.cos(q[i])*np.cos(alfa[i]), -np.cos(q[i])*np.sin(alfa[i]), a[i]*np.sin(q[i])],
                      [0, np.sin(alfa[i]), np.cos(alfa[i]), d[i]],
                      [0, 0, 0, 1]])
        
        R_3_0 = R_3_0*A[0:3,0:3]
    
    R_6_0 = POSE[0:3, 0:3]

    R_6_3 = (np.transpose(R_3_0)).dot(R_6_0)
    #print(R_6_3)
    #Cálculo q5
    
    c5 = R_6_3.item((2,2))
    s5 = np.sqrt(1-c5**2)
    q[4] = np.arctan2(conf[2]*s5, c5)
    
    #Para salvar la indeterminación de c5 = 1 (az' = 1) que hace que ax' = ay' = 0 y sz' = nz' = 0 (ortogonal) hacemos:
    
    if (np.absolute(R_6_3.item((1,2))) < 1e-12) and (np.absolute(R_6_3.item((0,2))) < 1e-12) :
        
        q[3] = q4_actual #Cálculo de q4
        q[5] = np.arctan2(R_6_3.item((1,0)), R_6_3.item((0,0))) - q4_actual #Cálculo de q6
    else:
        #Cálculo de q4

        s4 = conf[2]*R_6_3.item((1,2))
        c4 = conf[2]*R_6_3.item((0,2))
        q[3] = np.arctan2(s4,c4)

        #Cálculo de q6

        s6 = conf[2]*R_6_3.item((2,1))
        c6 = -conf[2]*R_6_3.item((2,0))
        q[5] = np.arctan2(s6,c6)

        
    #np.set_printoptions(suppress=True) #Suprime la notación científica
    
    return (q)

In [4]:
#Pruebo el IRB140

q_slide50 = np.transpose(np.array([0, -np.pi/2, np.pi, 0, np.pi/2, 0]))
#print(q_slide50*180/np.pi)
a = np.array([70, 360, 0, 0, 0, 0])
d = np.array([0, 0, 0, 380, 0, 0])
alfa = np.array([-np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0])

[POSE,conf] = pcd_IRB140(q_slide50, a, d, alfa)

print("Veremos como queda la terna 6 en términos de la terna 0")
print(POSE,type(POSE),"\n")

print("Brazo adelante, codo negativo, muñeca positiva")
print(conf, type(conf), "\n")

q = pci_IRB140(POSE, a, d, alfa, conf, 0, 0)
print("El vector de posiciones articulares es: \n")
print(type(q),"\n",np.transpose(np.matrix(q)))

Veremos como queda la terna 6 en términos de la terna 0
[[ -1.  -0.   0. 450.]
 [  0.   1.   0.   0.]
 [ -0.   0.  -1. 360.]
 [  0.   0.   0.   1.]] <class 'numpy.matrix'> 

Brazo adelante, codo negativo, muñeca positiva
[ 1. -1.  1.] <class 'numpy.ndarray'> 

El vector de posiciones articulares es: 

<class 'numpy.ndarray'> 
 [[ 0.        ]
 [-1.57079633]
 [ 3.14159265]
 [ 0.        ]
 [ 1.57079633]
 [ 0.        ]]


##### Pudimos chequear que nos devuelve las posiciones articulares que esperamos

In [5]:
#Chequeamos ahora con otro ejemplo donde saltó la ficha que algo faltaba ajustar.. 

q = [0, -np.pi/2, np.pi, 0, 0, np.pi/2]
[POSE,conf] = pcd_IRB140(q, a, d, alfa)
print(POSE)
qinv = pci_IRB140(POSE,a,d,alfa,conf,0,0)
print("\n",np.transpose(np.matrix(qinv)))



[[ -0.  -0.   1. 450.]
 [  1.   0.   0.   0.]
 [  0.   1.   0. 360.]
 [  0.   0.   0.   1.]]

 [[ 0.        ]
 [-1.57079633]
 [ 3.14159265]
 [ 0.        ]
 [ 0.        ]
 [ 1.57079633]]


In [7]:
qinv = pci_IRB140(POSE,a,d,alfa,conf,0,0.1)
print("\n",np.transpose(np.matrix(qinv)))
[POSE2,conf] = pcd_IRB140(qinv, a, d, alfa)
print(POSE2)


 [[ 0.        ]
 [-1.57079633]
 [ 3.14159265]
 [ 0.1       ]
 [ 0.        ]
 [ 1.47079633]]
[[ -0.  -0.   1. 450.]
 [  1.   0.   0.   0.]
 [ -0.   1.   0. 360.]
 [  0.   0.   0.   1.]]
